In [ ]:
import matplotlib.pyplot as plt
# import seaborn as sns

import pandas as pd
import numpy as np

import os
from tqdm import tqdm
import json
import shutil
import pydicom

from PIL import Image
from PIL import ImageDraw
import glob

In [ ]:
#Original
image_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/ART/02_01_0001_0049.png'
img=Image.open(image_path)

img

In [ ]:
#Draw Circle


tr_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320]
te_list =  [321, 322, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 347, 348, 349, 350, 351, 352, 353, 354, 355, 357, 358, 359, 360]

pt_num = list(set(tr_list)|set(te_list))
#pt_num=te_list

LABEL_path_train = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/라벨링데이터/'
file_path_train = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/원천데이터/'
LABEL_path_test = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/2.Validation/라벨링데이터/'
file_path_test = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/2.Validation/원천데이터/'


#LEFT_ART
for fn in pt_num: 
    x_leftMass_art = []
    y_leftMass_art = []
    fn = format(fn,'04d')
    folder = '02_01_'+fn
    num = 0

    if int(fn) < 321: #train data foler
        path = file_path_train+folder+'/'+folder+'_ART/'
        file_names = os.listdir(path)
        ppl_ARTlabel_path = LABEL_path_train + folder +'/' + folder + '_ART.json'
        num=len(file_names) #the number of each patient's total CT images
    else: #val data folder
        path = file_path_test+folder+'/'+folder+'_ART/'
        file_names = os.listdir(path)
        ppl_ARTlabel_path = LABEL_path_test + folder +'/' + folder + '_ART.json'
        num=len(file_names) #the number of each patient's total CT images
    print(path)
    with open(ppl_ARTlabel_path) as f: 
        ART_json = json.load(f)
    length = len(ART_json['roi_list']['left mass']) #keyword / ex. left mass, right mass2, left kidney
    z_list = []
    for i in range(length): 
        z_list.append(ART_json['roi_list']['left mass'][i]['z']) #extract 'z' coordinate of mass images
    z_list = list(set(z_list))
    x_list = []
    y_list = []
    
    for j in z_list:
        for i in range(length): 
            if ART_json['roi_list']['left mass'][i]['z'] == j:
                x_list.append(ART_json['roi_list']['left mass'][i]['x'])
                y_list.append(ART_json['roi_list']['left mass'][i]['y'])
        x_leftMass_art.append(x_list)
        y_leftMass_art.append(y_list)
        x_list=[]
        y_list=[]


    file_names = glob.glob(path+'/*')
    index = 0
    if len(z_list) > 0:
        for j in z_list:
            
            for w in file_names: 
                header = pydicom.dcmread(w, stop_before_pixels = True)
                h_num = header.InstanceNumber 
                if h_num == num-j:#find the number of png
                    str = ''
                    if int(fn)<321:
                        str=w[111:115]
                    else:
                        str=w[113:117]
                    print(str)
                    image_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/ART/{}.png'.format(folder+'_'+str)
                    img=Image.open(image_path).convert('RGB')
                    draw=ImageDraw.Draw(img,'RGBA')
                    
                    r=0.1 #radius
                    n = len(x_leftMass_art[index])
                    #print(n)
                    
                    for i in range(n):
                        x = x_leftMass_art[index][i]
                        y = y_leftMass_art[index][i]
                        draw.ellipse((x-r, y-r, x+r, y+r), fill=(255,0,0,20)) #the fourth parameter of fill is related to the opacity
                    index=index+1
                    img.save('/home/ncp/workspace/blocks1/kidneyData_windowing_massColor/ART/{}.png'.format(folder+'_'+str),'png')
                